<a href="https://colab.research.google.com/github/abhinav9629/JPEGUP/blob/main/Jpeg2SRJpeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.models import vgg19
from PIL import Image
import albumentations as A
from torchvision.utils import save_image
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim

In [63]:
# import shutil
# import os

# source = '/content/drive/MyDrive/SR_Data/DIV2K_train_HR'
# destination = '/content/test_images'

# # gather all files
# filess = []
# allfiles = os.listdir(source)
# for files in allfiles:
#   if files.endswith("x4.png") and len(filess) < 21:
#     filess.append(files)

# # iterate on all files to move them to destination folder
# for f in filess:
#     src_path = os.path.join(source, f)
#     dst_path = os.path.join(destination, f)
#     shutil.copy(src_path, dst_path)

In [80]:
device = "cuda" if torch.cuda.is_available() else "cpu"
load_model = False
save_model = True
chkpt_gen = "/content/drive/MyDrive/SR_Data/SR_Checkpoints/generator.pth.tar"
chkpt_disc = "/content/drive/MyDrive/SR_Data/SR_Checkpoints/discriminator.pth.tar"
learning_rate = 1e-4
epochs = 100
batch_size = 16
num_workers = 4
high_res_size = 96
low_res_size = 24
img_channels = 3

In [81]:
high_res_transform = A.Compose(
    [
        A.Normalize(mean=[0.5 for _ in range(img_channels)], std=[0.5 for _ in range(img_channels)]),
        ToTensorV2(),
    ]
)

low_res_transform = A.Compose(
    [
        A.Resize(width = low_res_size, height = low_res_size, interpolation = Image.BICUBIC),
        A.Normalize(
            mean = [0 for _ in range(img_channels)], std = [1 for _ in range(img_channels)]
        ),
        ToTensorV2(),
    ]
)

both_transform = A.Compose(
    [
          A.RandomCrop(width = high_res_size , height = high_res_size),
          A.HorizontalFlip(p = 0.5),
          A.RandomRotate90(p = 0.5),
    ]
)

test_transform = A.Compose(
    [
        A.Normalize(
            mean = [0 for _ in range(img_channels)], std = [1 for _ in range(img_channels)]
        ),
        ToTensorV2(),
    ]
)

In [82]:
class ImageDataset():
  def __init__(self, path_dir):
    self.data = []
    self.target = []
    self.path_dir = path_dir

    for files in os.listdir(self.path_dir):
      if files.endswith("x4.png"):
        self.data.append(files)
      else:
        self.target.append(files)

    self.data = sorted(self.data)
    self.target = sorted(self.target)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    label_name = self.data[index]
    image_name = self.target[index]
    image = np.asarray(Image.open(os.path.join(self.path_dir,image_name)))
    image = both_transform(image=image)["image"]
    high_res = high_res_transform(image = image)["image"]
    low_res = low_res_transform(image = image)["image"]

    return low_res, high_res

In [83]:
class C_Block(nn.Module):
  def __init__(self, in_ch, out_ch, kernel_size, stride, padding, disc = False, use_activation = True, use_batchnorm = True ):
     super().__init__()
     self.convnet = nn.Conv2d(in_ch, out_ch, kernel_size = kernel_size, stride = stride, padding = padding, bias = not use_batchnorm)
     self.bn = nn.BatchNorm2d(out_ch) if use_batchnorm else nn.Identity()
     self.act = nn.LeakyReLU(0.2, inplace= True) if disc else nn.PReLU(num_parameters = out_ch)
     self.use_activation = use_activation

  def forward(self, input):
    x = self.convnet(input)
    x = self.bn(x)
    return self.act(x) if self.use_activation else x

In [84]:
class US_Block(nn.Module):
  def __init__(self, in_ch, scale):
    super().__init__()
    self.convnet = nn.Conv2d(in_ch, in_ch*scale**2, kernel_size = 3, stride = 1, padding=1)
    self.ps = nn.PixelShuffle(scale)
    self.act = nn.PReLU(num_parameters = in_ch)

  def forward(self, input):
    x = self.convnet(input)
    x = self.ps(x)
    return self.act(x)

In [85]:
class ResidualNet_Block(nn.Module):
  def __init__(self, in_ch):
    super().__init__()
    self.block1 = C_Block(in_ch, in_ch, kernel_size = 3, stride = 1, padding = 1 )
    self.block2 = C_Block(in_ch, in_ch, kernel_size = 3, stride = 1, padding = 1, use_activation = False)

  def forward(self, input):
    x = self.block1(input)
    out = self.block2(x)
    return out + input

In [86]:
class Generator(nn.Module):
  def __init__(self, in_ch = 3, num_ch = 64, num_blocks = 16):
    super().__init__()
    self.init_convnet = C_Block(in_ch, num_ch, kernel_size = 9, stride = 1, padding = 4, use_batchnorm=False)
    self.residual = nn.Sequential(
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
        ResidualNet_Block(num_ch),
    )
    self.convnet = C_Block(num_ch, num_ch, kernel_size = 3, stride = 1, padding = 1, use_activation = False)
    self.upsamples = nn.Sequential(
        US_Block(num_ch,scale = 2),
        US_Block(num_ch,scale = 2),
    )
    self.final_convnet = nn.Conv2d(num_ch, in_ch, kernel_size = 9, stride = 1, padding = 4)

  def forward(self, input):
    initial = self.init_convnet(input)
    x = self.residual(initial)
    x = self.convnet(x) + initial
    x = self.upsamples(x)
    out = self.final_convnet(x)
    return torch.tanh(out)


In [87]:
class Discriminator(nn.Module):
  def __init__(self, in_ch = 3, features = 64):
    super().__init__()
    self.convnet = nn.Sequential(
        C_Block(in_ch, features, kernel_size = 3, stride = 1, padding = 1, disc = True, use_activation = True, use_batchnorm = False ),
        C_Block(features, features, kernel_size = 3, stride = 2, padding = 1, disc = True, use_activation = True, use_batchnorm = True ),
        C_Block(features, features*2, kernel_size = 3, stride = 1, padding = 1, disc = True, use_activation = True, use_batchnorm = True ),
        C_Block(features*2, features*2, kernel_size = 3, stride = 2, padding = 1, disc = True, use_activation = True, use_batchnorm = True ),
        C_Block(features*2, features*4, kernel_size = 3, stride = 1, padding = 1, disc = True, use_activation = True, use_batchnorm = True ),
        C_Block(features*4, features*4, kernel_size = 3, stride = 2, padding = 1, disc = True, use_activation = True, use_batchnorm = True ),
        C_Block(features*4, features*8, kernel_size = 3, stride = 1, padding = 1, disc = True, use_activation = True, use_batchnorm = True ),
        C_Block(features*8, features*8, kernel_size = 3, stride = 2, padding = 1, disc = True, use_activation = True, use_batchnorm = True ),
    )
    self.dense = nn.Sequential(
        nn.AdaptiveAvgPool2d((6,6)),
        nn.Flatten(),
        nn.Linear(512*6*6, 1024),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Linear(1024,1),
    )

  def forward(self, input):
    x = self.convnet(input)
    out = self.dense(x)
    return out

In [88]:
# def test():
#   low_res = 24
#   with torch.cuda.amp.autocast():
#     x = torch.randn((5, 3, low_res,low_res))
#     gen = Generator()
#     disc = Discriminator()
#     gen_out = gen(x)
#     disc_out = disc(gen_out)
#     print(gen_out.shape)
#     print(disc_out.shape)
# test()

In [89]:
class VGGLoss(nn.Module):
  def __init__(self):
    super().__init__()
    self.vgg = vgg19(pretrained = True).features[:36].eval().to(device)
    self.loss = nn.MSELoss()

    for param in self.vgg.parameters():
      param.requires_grad = False

  def forward(self, input, target):
    vgg_inp_feat  = self.vgg(input)
    vgg_tar_feat  = self.vgg(target)
    return self.loss(vgg_inp_feat, vgg_tar_feat)

In [90]:
def save_chkpt(model, optimizer, path="my_checkpoint.pth.tar"):
  print("=> Saving Checkpoint")
  checkpoint = {
      "state_dict" : model.state_dict(),
      "optimizer" : optimizer.state_dict()
  }
  torch.save(checkpoint,path)

def load_chkpt(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=device)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

In [91]:
def plot_examples(low_res_folder, gen):
    files = os.listdir(low_res_folder)
    gen.eval()
    for fil in files:
        image = Image.open(os.path.join(low_res_folder,fil))
        with torch.no_grad():
            upscaled_img = gen(
                 test_transform(image=np.asarray(image))["image"]
                .unsqueeze(0)
                .to(device)
            )
        save_image(upscaled_img * 0.5 + 0.5, f"/content/saved/{fil}")
    gen.train()

In [92]:
train_path  = "/content/drive/MyDrive/SR_Data/DIV2K_train_HR"
dataset = ImageDataset(path_dir = train_path)
loader = DataLoader(dataset,batch_size = batch_size, shuffle = True, pin_memory = True, num_workers=2)

In [93]:
generator = Generator(in_ch = 3).to(device)
discriminator = Discriminator(in_ch = 3).to(device)
opt_gen = optim.Adam(generator.parameters(), lr = learning_rate, betas = (0.9,0.999))
opt_disc = optim.Adam(discriminator.parameters(), lr = learning_rate, betas = (0.9,0.999))
mse_loss = nn.MSELoss()
bce_loss = nn.BCEWithLogitsLoss()
vgg_loss = VGGLoss()

In [94]:
if load_model:
  load_chkpt(chkpt_gen, generator, opt_gen, learning_rate)
  load_chkpt(chkpt_disc, discriminator, opt_disc, learning_rate)

In [95]:
for epoch in range(epochs):
  loop = tqdm(loader, leave= True)
  loop.set_description(f"Epoch {epoch}:")

  for batch_idx, (low_res, high_res) in enumerate(loop):
    low_res = low_res.to(device)
    high_res = high_res.to(device)

    fake = generator(low_res)
    # disc_real = discriminator(high_res)
    # disc_fake = discriminator(fake.detach())
    # disc_loss_real = bce_loss(disc_real, torch.ones_like(disc_real) - 0.1 * torch.rand_like(disc_real))
    # disc_loss_fake = bce_loss(disc_fake, torch.zeros_like(disc_fake))
    # disc_loss  = disc_loss_real + disc_loss_fake

    # opt_disc.zero_grad()
    # disc_loss.backward()
    # opt_disc.step()

    l2_loss = mse_loss(fake,high_res)
    #disc_fake = discriminator(fake)
    # gen_gan_loss = 1e-3*bce_loss(disc_fake, torch.ones_like(disc_fake))
    # vgg19_loss = 0.006*vgg_loss(fake,high_res)
    #gen_loss  = gen_gan_loss + vgg19_loss
    gen_loss = l2_loss

    opt_gen.zero_grad()
    gen_loss.backward()
    opt_gen.step()

    if batch_idx % 200 == 0:
      plot_examples("/content/test_images",generator)


  if save_model and epoch%5 == 0:
    print(f"Epoch: {epoch}, LossG_L2: {gen_loss}")
    save_chkpt(generator, opt_gen, path = chkpt_gen)
    save_chkpt(discriminator, opt_disc, path = chkpt_disc)




Epoch 0:: 100%|██████████| 50/50 [01:48<00:00,  2.16s/it]


Epoch: 0, LossG_L2: 0.045795682817697525
=> Saving Checkpoint
=> Saving Checkpoint


Epoch 5:: 100%|██████████| 50/50 [01:46<00:00,  2.12s/it]


Epoch: 5, LossG_L2: 0.027267929166555405
=> Saving Checkpoint
=> Saving Checkpoint


Epoch 10:: 100%|██████████| 50/50 [01:48<00:00,  2.18s/it]


...

Epoch 90:: 100%|██████████| 50/50 [01:52<00:00,  2.25s/it]


Epoch: 90, LossG_L2: 0.02515692636370659
=> Saving Checkpoint
=> Saving Checkpoint


Epoch 95:: 100%|██████████| 50/50 [01:54<00:00,  2.30s/it]


Epoch: 95, LossG_L2: 0.02725205384194851
=> Saving Checkpoint
=> Saving Checkpoint


Epoch 99:: 100%|██████████| 50/50 [01:53<00:00,  2.26s/it]


In [96]:
save_chkpt(generator, opt_gen, path = "/content/drive/MyDrive/SR_Data/SR_Checkpoints/Epoch200/generator.pth.tar")
save_chkpt(discriminator, opt_disc, path = "/content/drive/MyDrive/SR_Data/SR_Checkpoints/Epoch200/discriminator.pth.tar")

=> Saving Checkpoint
=> Saving Checkpoint
